# Token Classification

This generic task encompasses any problem that can be formulated as “attributing a label to each token in a sentence,” such as:

* **Named entity recognition (NER)**: Find the entities (such as persons, locations, or organizations) in a sentence. This can be formulated as attributing a label to each token by having one class per entity and one class for “no entity.”

* **Part-of-speech tagging (POS)**: Mark each word in a sentence as corresponding to a particular part of speech (such as noun, verb, adjective, etc.).

* **Chunking**: Find the tokens that belong to the same entity. This task (which can be combined with POS or NER) can be formulated as attributing one label (usually B-) to any tokens that are at the beginning of a chunk, another label (usually I-) to tokens that are inside a chunk, and a third label (usually O) to tokens that don’t belong to any chunk.


Of course, there are many other types of token classification problem; those are just a few representative examples. In this section, we will fine-tune a model (BERT) on a NER task.

## 1. Preparing the data

In this section we will use the CoNLL-2003 dataset, which contains news stories from Reuters.

### 1.1. The CoNLL-2003 dataset

In [17]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


In [18]:
from datasets import load_dataset, DatasetDict

train = load_dataset(
    "parquet",
    data_files="hf://datasets/eriktks/conll2003@refs/convert/parquet/conll2003/train/*.parquet",
    split="train",
)

valid = load_dataset(
    "parquet",
    data_files="hf://datasets/eriktks/conll2003@refs/convert/parquet/conll2003/validation/*.parquet",
    split="train",
)

test = load_dataset(
    "parquet",
    data_files="hf://datasets/eriktks/conll2003@refs/convert/parquet/conll2003/test/*.parquet",
    split="train",
)

data_files = {
    "train": train,
    "validation": valid,
    "test": test
}

raw_datasets = DatasetDict({
    "train": train,
    "validation": valid,
    "test": test
})

In [19]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [20]:
raw_datasets["train"][:2]

{'id': ['0', '1'],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.'],
  ['Peter', 'Blackburn']],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7], [22, 22]],
 'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0], [11, 12]],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0], [1, 2]]}

In particular, we can see the dataset contains labels for the three tasks we mentioned earlier: NER, POS, and chunking. A big difference from other datasets is that the input texts are not presented as sentences or documents, but lists of words.

Since we want to perform named entity recognition, we will look at the NER tags:



In [21]:
raw_datasets["train"][0]["ner_tags"]


[3, 0, 7, 0, 0, 0, 7, 0, 0]

Those are the labels as integers ready for training, but they’re not necessarily useful when we want to inspect the data. Like for text classification, we can access the correspondence between those integers and the label names by looking at the `features` attribute of our dataset:



In [22]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']))

So this column contains elements that are lists of `ClassLabels`. The type of the elements of the list is in the feature attribute of this `ner_feature`, and we can access the list of names by looking at the `names` attribute of that feature:



In [23]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

* `O` means the word doesn’t correspond to any entity.

* `B-PER/I-PER` means the word corresponds to the beginning of/is inside a person entity.

* `B-ORG/I-ORG` means the word corresponds to the beginning of/is inside an organization entity.

* `B-LOC/I-LOC` means the word corresponds to the beginning of/is inside a location entity.

* `B-MISC/I-MISC` means the word corresponds to the beginning of/is inside a miscellaneous entity.

### 1.2. Processing the data

As usual, our texts need to be converted to token IDs before the model can make sense of them. To begin, let’s create our `tokenizer` object. We’ll start by downloading and caching the associated tokenizer:

In [24]:
from transformers import AutoTokenizer

checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

To tokenize a pre-tokenized input, we can use our `tokenizer` as usual and just add `is_split_into_words=True`:



In [25]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs)
print(inputs.tokens())

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['[CLS]', 'EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'la', '##mb', '.', '[SEP]']


The word `lamb` was tokenized into two subwords, `la` and `##mb`. This introduces a mismatch between our inputs and the labels: the list of labels has only 9 elements, whereas our input now has 12 tokens. Accounting for the special tokens is easy (we know they are at the beginning and the end), but we also need to make sure we align all the labels with the proper words.

Fortunately, because we’re using a fast tokenizer we have access to the 🤗 Tokenizers superpowers, which means we can easily map each token to its corresponding word.

In [26]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

With a tiny bit of work, we can then expand our label list to match the tokens. The first rule we’ll apply is that special tokens get a label of `-100`. This is because by default `-100` is an index that is ignored in the loss function we will use (cross entropy). Then, each token gets the same label as the token that started the word it’s inside, since they are part of the same entity. For tokens inside a word but not at the beginning, we replace the `B-` with `I-` (since the token does not begin the entity):



In [27]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_id = None
    for word_id in word_ids:
        # New word
        if word_id != current_id:
            current_id = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        # Special token
        elif word_id is None:
            new_labels.append(-100)
        # Same id as previous token
        else:
            label = labels[word_id]
            if label % 2 == 1:
                label += 1
            new_labels.append(label)
    return new_labels

To preprocess our whole dataset, we need to tokenize all the inputs and apply `align_labels_with_tokens()` on all the labels. To take advantage of the speed of our fast tokenizer, it’s best to tokenize lots of texts at the same time, so we’ll write a function that processes a list of examples and use the `Dataset.map()` method with the option `batched=True`. The only thing that is different from our previous example is that the `word_ids()` function needs to get the index of the example we want the word IDs of when the inputs to the tokenizer are lists of texts (or in our case, list of lists of words), so we add that too:



In [28]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, label in enumerate(all_labels):
        word_id = tokenized_inputs.word_ids(i)
        new_label = align_labels_with_tokens(label, word_id)
        new_labels.append(new_label)
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

Note that we haven’t padded our inputs yet; we’ll do that later, when creating the batches with a data collator.



We can now apply all that preprocessing in one go on the other splits of our dataset:



In [29]:
tokenized_dataset = raw_datasets.map(
    tokenize_and_align_labels, 
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

## 2. Fine-tuning the model with the Trainer API

The actual code using the `Trainer` will be the same as before; the only changes are the way the data is collated into a batch and the metric computation function.



### 2.1. Data collation

We can’t just use a `DataCollatorWithPadding` because that only pads the inputs (input IDs, attention mask, and token type IDs). Here our labels should be padded the exact same way as the inputs so that they stay the same size, using `-100` as a value so that the corresponding predictions are ignored in the loss computation.

This is all done by a `DataCollatorForTokenClassification`. Like the `DataCollatorWithPadding`, it takes the tokenizer used to preprocess the inputs:



In [30]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

To test this on a few samples, we can just call it on a list of examples from our tokenized training set:



In [31]:
batch = data_collator([tokenized_dataset["train"][i] for i in range(5)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    5,    6,    6,    6,    0,    0,    0,    0,    0, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100],
        [-100,    0,    3,    4,    0,    0,    0,    0,    0,    0,    7,    0,
            0,    0,    0,    0,    0,    7,    0,    0,    0,    0,    0,    0,
            0,    0,    0

### 2.2. Metrics

To have the `Trainer` compute a metric every epoch, we will need to define a `compute_metrics()` function that takes the arrays of predictions and labels, and returns a dictionary with the metric names and values.



The traditional framework used to evaluate token classification prediction is `seqeval`. To use this metric, we first need to install the `seqeval` library:



In [32]:
!pip install seqeval

We can then load it via the `evaluate.load()` function.

In [33]:
import evaluate

metric = evaluate.load('seqeval')

This `compute_metrics()` function first takes the argmax of the logits to convert them to predictions (as usual, the logits and the probabilities are in the same order, so we don’t need to apply the softmax). Then we have to convert both labels and predictions from integers to strings. We remove all the values where the label is `-100`, then pass the results to the `metric.compute()` method:



In [43]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    # Remove special tokens and convert to labels
    true_labels = [
        [label_names[i] for i in label if i != -100] for label in labels
    ]
    true_predictions = [
        [label_names[l] for l, p in zip(label, prediction) if l != -100] 
        for label, prediction in zip(labels, predictions)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"]
    }


Now that this is done, we are almost ready to define our `Trainer`. We just need a model to fine-tune!



### 2.3. Defining the model

Since we are working on a token classification problem, we will use the `AutoModelForTokenClassification` class. The main thing to remember when defining this model is to pass along some information on the number of labels we have. The easiest way to do this is to pass that number with the `num_labels` argument, but if we want a nice inference widget working like the one we saw at the beginning of this section, it’s better to set the correct label correspondences instead.



They should be set by two dictionaries, `id2label` and `label2id`, which contain the mappings from ID to label and vice versa:



In [44]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

In [45]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id
)

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

BertForTokenClassification LOAD REPORT from: bert-base-cased
Key                                        | Status     | 
-------------------------------------------+------------+-
bert.pooler.dense.bias                     | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
bert.pooler.dense.weight                   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized beca

Creating the model issues a warning that some weights were not used (the ones from the pretraining head) and some other weights are randomly initialized (the ones from the new token classification head), and that this model should be trained. We will do that in a minute, but first let’s double-check that our model has the right number of labels:

In [46]:
model.config.num_labels

9

### 2.4. Fine-tuning the model

In [47]:
from huggingface_hub import notebook_login

notebook_login()

In [48]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
    logging_strategy='epoch'
)

In [49]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer
)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.139672,0.070630,1.000000,1.000000,1.000000,1.000000
2,0.042364,0.069165,1.000000,1.000000,1.000000,1.000000
3,0.022372,0.067698,1.000000,1.000000,1.000000,1.000000
4,0.011832,0.078232,1.000000,1.000000,1.000000,1.000000
5,0.006362,0.076854,1.000000,1.000000,1.000000,1.000000


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=8780, training_loss=0.044520623591603345, metrics={'train_runtime': 952.8878, 'train_samples_per_second': 73.676, 'train_steps_per_second': 9.214, 'total_flos': 1535504982265242.0, 'train_loss': 0.044520623591603345, 'epoch': 5.0})

Note that while the training happens, each time the model is saved (here, every epoch) it is uploaded to the Hub in the background. This way, you will be able to to resume your training on another machine if necessary.



Once the training is complete, we use the `push_to_hub()` method to make sure we upload the most recent version of the model:



In [50]:
trainer.push_to_hub(commit_message="Uploading final version of BERT fine-tuned for NER tasks")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ned-ner/training_args.bin: 100%|##########| 5.14kB / 5.14kB            

  ...ned-ner/model.safetensors:   8%|7         | 33.5MB /  431MB            

CommitInfo(commit_url='https://huggingface.co/arraypowerplay/bert-finetuned-ner/commit/95b9e8b235c8a796a88ddb0caa51a622da37417b', commit_message='Uploading final version of BERT fine-tuned for NER tasks', commit_description='', oid='95b9e8b235c8a796a88ddb0caa51a622da37417b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/arraypowerplay/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='arraypowerplay/bert-finetuned-ner'), pr_revision=None, pr_num=None)

If you want to dive a bit more deeply into the training loop, we will now show you how to do the same thing using 🤗 Accelerate.



## 3. A custom training loop

### 3.1. Preparing everything for training


First we need to build the `DataLoaders` from our datasets. We’ll reuse our `data_collator` as a `collate_fn` and shuffle the training set, but not the validation set:



In [51]:
import torch

train_dataloader = torch.utils.data.DataLoader(
    tokenized_dataset["train"],
    batch_size=16,
    collate_fn=data_collator,
    shuffle=True
)

eval_dataloader = torch.utils.data.DataLoader(
    tokenized_dataset["validation"],
    batch_size=16,
    collate_fn=data_collator,
    shuffle=False
)

Next we reinstantiate our model, to make sure we’re not continuing the fine-tuning from before but starting from the BERT pretrained model again:



In [52]:
model = AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

BertForTokenClassification LOAD REPORT from: bert-base-cased
Key                                        | Status     | 
-------------------------------------------+------------+-
bert.pooler.dense.bias                     | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
bert.pooler.dense.weight                   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
classifier.weight                          | MISSING    | 
classifier.bias                            | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized beca

Then we will need an optimizer. We’ll use the classic `AdamW`, which is like `Adam`, but with a fix in the way weight decay is applied:



In [53]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

Once we have all those objects, we can send them to the `accelerator.prepare()` method:



In [54]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Now that we have sent our `train_dataloader` to `accelerator.prepare()`, we can use its length to compute the number of training steps. Remember that we should always do this after preparing the dataloader, as that method will change its length. We use a classic linear schedule from the learning rate to 0:



In [55]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_step_per_epoch = len(train_dataloader)
num_training_step = num_train_epochs * num_update_step_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_step
)

Lastly, to push our model to the Hub, we will need to create a `Repository` object in a working folder. First log in to Hugging Face, if you’re not logged in already. We’ll determine the repository name from the model ID we want to give our model (feel free to replace the `repo_name` with your own choice; it just needs to contain your username, which is what the function `get_full_repo_name()` does):



In [57]:
!pip install huggingface_hub

In [59]:
from huggingface_hub import HfApi, create_repo, get_full_repo_name

model_name = "bert-finetuned-ner-accelerate"
repo_name = get_full_repo_name(model_name)
create_repo(repo_name, exist_ok=True)

api = HfApi()
repo_name

'arraypowerplay/bert-finetuned-ner-accelerate'

Then we can clone that repository in a local folder. If it already exists, this local folder should be an existing clone of the repository we are working with:



In [60]:
output_dir = "bert-finetuned-ner-accelerate"

We can now upload anything we save in `output_dir` by calling the `repo.push_to_hub()` method. This will help us upload the intermediate models at the end of each epoch.



### 3.2. Training loop

We are now ready to write the full training loop. To simplify its evaluation part, we define this `postprocess()` function that takes predictions and labels and converts them to lists of strings, like our `metric` object expects:



In [61]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()
    true_labels = [
        [label_names[i] for i in label if i != -100] for label in labels
    ]
    true_predictions = [
        [label_names[p] for p, l in zip(prediction, label) if l != -100] 
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions


Here’s the complete code for the training loop:



In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_step))

for epoch in range(num_train_epochs):
    model.train()
    for batch in train_dataloader:
        output = model(**batch)
        loss = output.loss
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            output = model(**batch)
        predictions = output.logits.argmax(dim=-1)
        labels = batch["labels"]
        # Pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)
        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)
        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch + 1}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )
    # Save and upload to Hugging Face
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        api.upload_folder(
            folder_path=output_dir,
            repo_id=repo_name,
            commit_message=f"Training in progress with epoch {epoch + 1}",
        )

  0%|          | 0/4390 [00:00<?, ?it/s]

epoch 1: {'precision': np.float64(0.9306630764052508), 'recall': np.float64(0.8909295956178508), 'f1': np.float64(0.9103629928389169), 'accuracy': 0.9808530052393006}


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

epoch 5: {'precision': np.float64(0.9495119488387749), 'recall': np.float64(0.9290301333772435), 'f1': np.float64(0.9391593841032043), 'accuracy': 0.9860628716077}


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...elerate/model.safetensors:   0%|          |  558kB /  431MB            

Then we grab the `unwrapped_model`, which is the base model we defined. The `accelerator.prepare()` method changes the model to work in distributed training, so it won’t have the `save_pretrained()` method anymore; the `accelerator.unwrap_model()` method undoes that step. Lastly, we call `save_pretrained()` but tell that method to use `accelerator.save()` instead of `torch.save()`.

## 4. Using the fine-tuned model


We’ve already shown you how you can use the model we fine-tuned on the Model Hub with the inference widget. To use it locally in a `pipeline`, you just have to specify the proper model identifier:



In [3]:
from transformers import pipeline

model_checkpoint = "arraypowerplay/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

[{'entity_group': 'PER',
  'score': np.float32(0.99951494),
  'word': 'Sylvain',
  'start': 11,
  'end': 18},
 {'entity_group': 'ORG',
  'score': np.float32(0.9993232),
  'word': 'Hugging Face',
  'start': 33,
  'end': 45},
 {'entity_group': 'LOC',
  'score': np.float32(0.99432725),
  'word': 'Brooklyn',
  'start': 49,
  'end': 57}]